## Projected Wasserstein metric for distribution comparison

Suppose we have observed $d$-dimensional $x_1, \cdots, x_M\in\mathbb{R}^d$ following the distribution of a random variable $X\sim p$. In addition, we  observe another group of observations $y_1, \cdots, y_N\in\mathbb{R}^d$ which follow the distribution of a random variable $Y\sim q$. Our target is to compute the distance between the two samples. To do so, we repeatedly carry out:


1. Randomly draw a unit vector $v\in\mathbb{R}^d$ ($\|v \|_2 = 1$)
2. Project the samples of $X$ onto this direction, i.e., compute vector inner product $r_i = \langle x_i, v\rangle$ for $i=1,\cdots,M$.
3. Project the samples of $Y$ onto this direction, i.e., compute vector inner product $s_j = \langle y_j, v\rangle$ for $j=1,\cdots, N$.
4. Compute the Wasserstein distance $D\big(\{r_i\}_{i=1}^M, \{s_j\}_{j=1}^N\big)$ between two one-dimensional data samples $\{r_i\}_{i=1}^M$ and $\{s_j\}_{j=1}^N$.

Step 1--4 are repeated multiple times and the average distance is computed as our final discrepancy measure. The details of Step 4 are provided below. 

### Computation of one-dimensional Wasserstein distance
In order to compute the Wasserstein distance between $\{r_i\}_{i=1}^M$ and $\{s_j\}_{j=1}^N$ on the real axis $\mathbb{R}$, the two samples are discretized firstly:

- Let $r_\max$ and $r_\min$ being the maximum and minimum value of $\{r_i\}_{i=1}^M$, respectively. Then each projected sample $r_i$ is inside the interval $[r_\min, r_\max]$. The interval is equally cut into 200 subintervals $\mathcal{I} = [r_\min, r_\max]=\cup_{a=1}^{200} \mathcal{I}_a$, with the $a$-th subinterval being
$$
\mathcal{I}_a = [r_\min + (a-1) \Delta_r,\, r_\min + a \Delta_r),\quad \text{ and } \quad
\Delta_r = (r_\max - r_\min)/200.
$$
Let $C(\mathcal{I}_a) = |\{r_i:\, r_i\in \mathcal{I}_a\}|$ be the number of projected samples inside the subinterval $\mathcal{I}_a$. Then 
$P_a = C(\mathcal{I}_a) / M$ is *proportion* of projected samples inside the subinterval $\mathcal{I}_a$.
In fact, $P_a$ is a discretized probability distribution for $\{r_i\}_{i=1}^M$.
- The same procedure is carried out for  $\{s_j\}_{j=1}^N$. Let $s_\max$ and $s_\min$ being the maximum and minimum value of $\{s_i\}_{i=1}^N$, respectively. The interval $[s_\min, s_\max]$ is equally cut into 200 subintervals $\mathcal{J} = [s_\min, s_\max]=\cup_{b=1}^{200} \mathcal{J}_b$, with the $b$-th subinterval being
$$
\mathcal{J}_b = [s_\min + (b-1) \Delta_s,\, s_\min + b \Delta_s),\quad \text{ and } \quad
\Delta_s = (s_\max - s_\min)/200.
$$
Let $C(\mathcal{J}_b) = |\{s_j:\, s_j\in \mathcal{J}_b\}|$ be the number of projected samples inside the subinterval $\mathcal{J}_b$. Then 
$Q_b = C(\mathcal{J}_b) / N$ is *proportion* of projected samples inside the subinterval $\mathcal{J}_b$.
In fact, $Q_b$ is a discretized probability distribution for $\{s_j\}_{j=1}^N$.

The Wasserstein distance is computed by matching the first distribution $\{P_a\}_{a=1}^{200}$ to the second 
$\{Q_a\}_{a=1}^{200}$. To do so, it tries to make minimal amount of adjustment to $\{P_a\}_{a=1}^{200}$, such that the first distribution looks exactly like the second $\{Q_a\}_{a=1}^{200}$. Obviously, the closer the two distributions, the less effort is required to match the two distribuitons.

To quantify the amount of effort, let $T_{ab}$ be the proportion of the first sample inside the subinterval $\mathcal{I}_a$ that should be moved (transported) to $\mathcal{J}_b$. 
- The cost of one such transportation is the distance between the two subintervals, i.e., between $\mathcal{I}_a$ and  $\mathcal{J}_b$. Let $L_{Ia} = r_\min + a \Delta_r$  and $L_{Jb} = s_\min + b \Delta_s$ be the upper bound of the subintervals. Then the transportation cost is computed by
$D_{ab} = |L_{Ia} - L_{Jb}|$.
- As all the transported samples come from the first distribution, we require the marginal condition $P_a = \sum_{b=1}^{200} T_{ab}$. That is, samples originated from $\mathcal{I}_a$ should have the proportion $P_a$ exactly. 
- On the other hand, we also require another marginal condition $Q_b = \sum_{a=1}^{200} T_{ab}$. This means that, after transportation, the first sample exactly looks like the second.

All of these enable us to formulate the problem as an optimal transportation problem. The transportation plan $T_{ab}$ for $a,b=1,\cdots, 200$ should satisfy the two mariginal conditions. At the same time, it should have minimal transportation cost. 
$$
\begin{align}
\min & \sum_{a,b=1}^{200} T_{ab}\cdot D_{ab}. \\
\text{subject to } & P_a = \sum_{b=1}^{200} T_{ab} \text{ and } Q_b = \sum_{a=1}^{200} T_{ab}.
\end{align}
$$
This optimization problem is solved by the python package [optimal transport](https://pot.readthedocs.io/en/stable/). Suppose $T_{ab}^*$ is the optimal solution. Then, our 
distance between $\{r_i\}_{i=1}^M$ and $\{s_j\}_{j=1}^N$  is computed as
$$
D\big(\{r_i\}_{i=1}^M, \{s_j\}_{j=1}^N\big) = \sum_{a,b=1}^{200} T^*_{ab}\cdot D_{ab}.
$$

More reference can be found at the [wiki page](https://en.wikipedia.org/wiki/Wasserstein_metric) and the [tutorial](http://www.stat.cmu.edu/~larry/=sml/Opt.pdf).

## Numerical Example

Basic setup:

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from astropy.io import fits
from scipy import stats
import wass


def readData(filePath):
    ffile = fits.open(filePath)
    u1 = ffile[1].data['u']
    g1 = ffile[1].data['g']
    r1 = ffile[1].data['r']
    i1 = ffile[1].data['i']
    z1 = ffile[1].data['z']
    ds1 = np.vstack((u1,g1,r1,i1,z1)).T
    sel = np.sum(np.abs(ds1) == 99,1) == 0
    data = ds1[sel,:]
    ffile.close()
    return data


Read two data sets from 
>      Zhou. et al.(2019) Deep ugrizY Imaging and DEEP2/3 Spectroscopy ....

We will compare the difference between the two datasets, `ds1` and `ds2`. 
The loaded numpy arrays contain the $ugriz$ magnitude with missing values removed.

In [3]:
ds1 = readData("./data/3D-HST_Terapix_Wide_Subaru_v1.fits")
ds2 = readData("./data/DEEP2_uniq_Terapix_Wide_Subaru_v1.fits")
wass.compareDensity(ds1, ds2)

100% (2000 of 2000) |####################| Elapsed Time: 0:00:07 Time:  0:00:07


(1.5455423473913257, 0.022624684995631698)

In [4]:
ds1 = readData("./data/DEEP2_uniq_Terapix_Subaru_v1.fits")
ds2 = readData("./data/DEEP2_uniq_Terapix_Wide_Subaru_v1.fits")
wass.compareDensity(ds1, ds2)

100% (2000 of 2000) |####################| Elapsed Time: 0:00:09 Time:  0:00:09


(0.03127851119548137, 0.00026472907337255625)